In [1]:
import random
import sympy
from py_ecc.bn128 import G1, G2, add, multiply, neg, pairing, FQ, FQ2, curve_order

In [2]:
G = G1
p = curve_order
F = lambda x: x % p

In [3]:
'''
witness: x
output: x^2 - 25 = 0

We should proof that we know x as x^2 = 25
w0 = 1 (constant), w1 = x, w2 = x^2
'''

# I. Arithmetization (R1CS Rank 1 Constraint System)

A = [
    [0, 0, 1],  # A1: x
]
B = [
    [1, 0, 0],  # B1: x
]
C = [
    [0, 0, 0],  # C1: x^2
]
K = [25]

N_CONSTRAINTS = 1
N_VARS = 3

print("R1CS ")
print(f"- {N_CONSTRAINTS} constraints")
print(f"- {N_VARS} vars -> vector")
print("A:", A)
print("B:", B)
print("C:", C)
print("K:", K)

R1CS 
- 1 constraints
- 3 vars -> vector
A: [[0, 0, 1]]
B: [[1, 0, 0]]
C: [[0, 0, 0]]
K: [25]


In [4]:
def check_r1cs(x, nb_constraint=2, nb_var=3):
    w = [1, F(x), F(x*x)]

    for i in range(nb_constraint):
        A_dot = sum(A[i][j] * w[j] for j in range(nb_var))
        B_dot = sum(B[i][j] * w[j] for j in range(nb_var))
        C_dot = sum(C[i][j] * w[j] for j in range(nb_var)) + K[i]

        print(f"Constraint {i+1}:  {A_dot} * {B_dot} =? {C_dot}")

        if A_dot * B_dot != C_dot:
            print("R1CS invalid")
            return False
    
    print("R1CS valid")
    return True

check_r1cs(4, N_CONSTRAINTS, N_VARS)
print("-----")
check_r1cs(5, N_CONSTRAINTS, N_VARS)
print("-----")
check_r1cs(7, N_CONSTRAINTS, N_VARS)

Constraint 1:  16 * 1 =? 25
R1CS invalid
-----
Constraint 1:  25 * 1 =? 25
R1CS valid
-----
Constraint 1:  49 * 1 =? 25
R1CS invalid


False

In [5]:
# II. Constraints to polynomial ( R1CS -> QAP Quadratic Arithmetic Program) 
# Transform linear constraints to a unique polynomial, verifiable with an single secret point

def lagrange_basis(i, n):
    X = sympy.symbols('X')
    basis = 1
    for j in range(1, n+1):
        if j != i:
            basis *= (X - j) / (i - j)
    return sympy.simplify(basis)

L = [lagrange_basis(i+1, N_CONSTRAINTS) for i in range(N_CONSTRAINTS)]

print("Constructed basic Lagrange polynomials.")

def poly_from_R1CS_vector(vec):
    """Build a polynomial (N_CONSTRAINTS-1) from L_i."""
    X = sympy.symbols('X')
    poly = sum(vec[i] * L[i] for i in range(N_CONSTRAINTS))
    return sympy.simplify(poly)

A_poly = [poly_from_R1CS_vector([A[i][j] for i in range(N_CONSTRAINTS)]) for j in range(N_VARS)]
B_poly = [poly_from_R1CS_vector([B[i][j] for i in range(N_CONSTRAINTS)]) for j in range(N_VARS)]
C_poly = [poly_from_R1CS_vector([C[i][j] for i in range(N_CONSTRAINTS)]) for j in range(N_VARS)]
K_poly = poly_from_R1CS_vector(K)

print("Polynomials A(t), B(t), C(t), K(t) generated.")

Constructed basic Lagrange polynomials.
Polynomials A(t), B(t), C(t), K(t) generated.


In [6]:
# III. Parameters setup

X = sympy.symbols('X')
tau = random.randint(1, p-1)
Z_poly = X - 1

print("tau =", tau)

H_poly = sympy.sympify(0)

tau = 7580713744811195365917788591969312535478868780054765097992422875725999893626


In [7]:
# IV. Proof gen

def proof(x):
    w = [1, F(x), F(x*x)]

    def eval_poly(poly_list):
        return F(sum(F(poly_list[j].subs(X, tau)) * w[j] for j in range(N_VARS)))

    At = F(int(eval_poly(A_poly)))
    Bt = F(int(eval_poly(B_poly)))
    Ct = F(int(eval_poly(C_poly)))
    Kt = F(int(K_poly.subs(X, tau)))
    Ht = F(int(H_poly.subs(X, tau)))
    Zt = F(int(Z_poly.subs(X, tau)))

    # Blinding factors
    r_blinding = random.randint(1, p-1)
    s_blinding = random.randint(1, p-1)

    # Commitments
    A_exp_proof = F(At + r_blinding * Zt)
    B_exp_proof = F(Bt + s_blinding * Zt)
    C_exp_proof = F((Ct + Kt + Ht * Zt) + At * s_blinding * Zt + Bt * r_blinding * Zt + r_blinding * s_blinding * Zt * Zt)

    return {
        "piA": Gexp(A_exp_proof),
        "A_exp_proof": A_exp_proof,
        "piB": Gexp(B_exp_proof),
        "B_exp_proof": B_exp_proof,
        "piC": Gexp(C_exp_proof),
        "C_exp_proof": C_exp_proof
    }

In [8]:
# G1_base and G2_base for group operations
G1_base = G1
G2_base = G2

alpha = random.randint(1, p-1)
beta = random.randint(1, p-1)

crs_alpha_G1 = multiply(G1_base, alpha)
crs_beta_G2 = multiply(G2_base, beta)
crs_tau_G1 = multiply(G1_base, tau)
crs_tau_sq_G1 = multiply(G1_base, pow(tau, 2, p))

CRS = {
    "alpha_G1": crs_alpha_G1,
    "beta_G2": crs_beta_G2,
    "tau_G1": crs_tau_G1,
    "tau_sq_G1": crs_tau_sq_G1
}

print("Generated CRS:", CRS)

A_tau_scalars = [F(int(p_val.subs(X, tau))) for p_val in A_poly]
B_tau_scalars = [F(int(p_val.subs(X, tau))) for p_val in B_poly]
C_tau_scalars = [F(int(p_val.subs(X, tau))) for p_val in C_poly]
H_tau_scalar = F(int(H_poly.subs(X, tau)))
K_tau_scalar = F(int(K_poly.subs(X, tau)))

Generated CRS: {'alpha_G1': (4094047428731351180972421914377888906411356481988170052485791436388756117108, 9373223367993117077048685678987064446699358158352720524454208039290726388428), 'beta_G2': ((6338241494702574034508852761785920324226186555142919221686930816845435803970, 9582317434588318179552370699876069519081395738896305452540697226527424614018), (8382541770053232797237644339902420963672103797341188563551122104091188733572, 21885275444439097451535234121393202986042177649086711165055649762534828611821)), 'tau_G1': (16360469890731549673251456200349531149414114096165353910403332906129246304221, 17338134137047354606185720438609063518402996597402313128492032196155694147306), 'tau_sq_G1': (18367008458142391600658391117648456907714706712740117632594880120369041071996, 152184451281808882617461384429121066579306485715225175513816996978313766609)}


In [9]:
Z_tau_scalar = F(int(Z_poly.subs(X, tau)))
Z_tau_G1 = multiply(G1_base, Z_tau_scalar)

A_tau_G1 = [multiply(G1_base, scalar) for scalar in A_tau_scalars]
B_tau_G2 = [multiply(G2_base, scalar) for scalar in B_tau_scalars] # Commitments for B_poly in G2
C_tau_G1 = [multiply(G1_base, scalar) for scalar in C_tau_scalars]

H_tau_G1 = multiply(G1_base, H_tau_scalar)

K_tau_G1 = multiply(G1_base, K_tau_scalar)

print("\nZ(tau)*G1:", Z_tau_G1)
print("A(tau)*G1:", A_tau_G1)
print("B(tau)*G2:", B_tau_G2)
print("C(tau)*G1:", C_tau_G1)
print("H(tau)*G1:", H_tau_G1)
print("K(tau)*G1:", K_tau_G1)


Z(tau)*G1: (13703120436540485848112035397275504244374792255341496536446294308042511528764, 19553526273760899990600286283438103093245015860600369871533104627074954654664)
A(tau)*G1: [None, None, (1, 2)]
B(tau)*G2: [((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531)), None, None]
C(tau)*G1: [None, None, None]
H(tau)*G1: None
K(tau)*G1: (20765039372871530718554589730410158162413780974122112544611863764810626751360, 2444183914824638066910831265243126275246160293098948571390980460351548298384)


In [10]:
# IV. Proof gen

def proof(x):
    w = [1, F(x), F(x*x)]

    def eval_poly_scalar(poly_list):
        return F(sum(F(poly_list[j].subs(X, tau)) * w[j] for j in range(N_VARS)))

    At_scalar = eval_poly_scalar(A_poly)
    Bt_scalar = eval_poly_scalar(B_poly)
    Ct_scalar = eval_poly_scalar(C_poly)

    r_blinding = random.randint(1, p-1)
    s_blinding = random.randint(1, p-1)

    A_exp_scalar = F(At_scalar + r_blinding * Z_tau_scalar)
    B_exp_scalar = F(Bt_scalar + s_blinding * Z_tau_scalar)
    
    C_exp_scalar = F((Ct_scalar + K_tau_scalar + H_tau_scalar * Z_tau_scalar) + 
                     At_scalar * s_blinding * Z_tau_scalar + 
                     Bt_scalar * r_blinding * Z_tau_scalar + 
                     r_blinding * s_blinding * Z_tau_scalar * Z_tau_scalar)

    piA_point = multiply(G1_base, A_exp_scalar) # Commitment for A in G1
    piB_point = multiply(G2_base, B_exp_scalar) # Commitment for B in G2
    piC_point = multiply(G1_base, C_exp_scalar) # Commitment for C in G1

    print(f"Proof generated for x = {x}")
    return {
        "piA": piA_point,
        "piB": piB_point,
        "piC": piC_point,
        "A_exp_scalar": A_exp_scalar,
        "B_exp_scalar": B_exp_scalar,
        "C_exp_scalar": C_exp_scalar
    }



In [11]:
# V. Verification

def verify(proof):
    piA = proof["piA"]
    piB = proof["piB"]
    piC = proof["piC"]

    # e(piB, piA) = e(G2_base, piC)
    # Checks the identity A_exp_scalar * B_exp_scalar = C_exp_scalar over the curve.
    lhs = pairing(piB, piA) 

    rhs = pairing(G2_base, piC) # Changed CRS["beta_G2"] to G2_base for correct identity check

    return lhs == rhs


In [12]:
# ==== Testing ====

print("\nTesting x = 5")
proof_test = proof(5)
print("Proof for x=5:", proof_test)
print("Valid proof (correct x=5)?", verify(proof_test))

print("\nTesting x = 9")
proof_test = proof(9)
print("Proof for x=9:", proof_test)
print("Valid proof (incorrect x=9)?", verify(proof_test))


Testing x = 5
Proof generated for x = 5
Proof for x=5: {'piA': (11384427879782514755352670072532079087447279612825070139144521895162175521739, 21062230902117105171590080038039288240454044819039145516669563506751697533832), 'piB': ((19096606326062964657604973547882774527734032526200428461379610937081580403770, 5538819959612365441882188758745394006563598615392547672969976212185877912204), (20131819418971288794397276302202266398098683285305140076514617075144064689873, 17903977010339717861695104316866355313617762973264503367521426060781092167782)), 'piC': (4980648223048694169700574869074276928251466493095362737005517196222050402289, 17279788129349095508640423293498241631696504999277746479179352464477486754515), 'A_exp_scalar': 12921315548633924989141632384794727631153008618021991236751958610159240860540, 'B_exp_scalar': 13850416934572509508141758598688807575792068036719499465434928476575188014090, 'C_exp_scalar': 8594448116089678073996438008778881670737429101391316523499036454076104158228